In [1]:
from typing import List, Tuple
import torch
import copy
import os
import sys

# custom
from Clotho.Clotho_Dataset import * # 데이터셋
from transformers import GPT2Tokenizer
from ClipCap_forAAC.CLIPCAP_forAAC import * # network
from Train import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

data_dir = './Clotho'

# PANNs를 써먹기 위해 prefix_size를 수정
audio_prefix_size = 15
semantic_prefix_size = 11 
prefix_size = audio_prefix_size + semantic_prefix_size

transformer_num_layers = {"audio_num_layers" : 4, "semantic_num_layers" : 4}
prefix_size_dict = {"audio_prefix_size" : audio_prefix_size, "semantic_prefix_size" : semantic_prefix_size}


# vocab_size===============
# Custom Tokeniezr1 : 10627
# Custom Tokeniezr2 : 6524
# Clotho Tokeniezr : 6427
# vocab_size===============

vocab_size = 6427
tokenizer_type = 'Custom'

tokenizer = tokenizer_Clotho(vocab_size)
model_param_name = 'model_Clotho_Custom_' + str(vocab_size) + ".pt"

TEST_BATCH_SIZE = 5
test_dataloader  = dataloader_ClothoDataset(tokenizer, data_dir, TEST_BATCH_SIZE, split = 'evaluation', prefix_size = prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)

torch.cuda.empty_cache()

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

model = get_ClipCap_AAC(tokenizer, 
                        vocab_size = vocab_size, Dataset = 'Clotho',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = False, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

get dataset...: 100%|███████████████████████| 1045/1045 [00:49<00:00, 21.13it/s]
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


use Clotho Tokenizer
audio feature's mapping network : num_head = 8 num_layers = 4
semantic feature ver's mapping network : num_head = 8 num_layers = 4
GPT2 freezing


In [2]:
model_path = './Trained_model_params/' + model_param_name
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [3]:
eval_model(model, test_dataloader, 31, 'test', True)

Eval...: 100%|██████████████████████████████| 1045/1045 [03:00<00:00,  5.78it/s]


loading annotations into memory...
0:00:00.006887
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!


PTBTokenizer tokenized 65471 tokens at 457904.82 tokens per second.
PTBTokenizer tokenized 10181 tokens at 127658.42 tokens per second.


{'testlen': 9137, 'reflen': 9932, 'guess': [9137, 8092, 7047, 6002], 'correct': [5375, 2116, 857, 287]}
ratio: 0.9199556987514176


Parsing reference captions
Parsing test captions


SPICE evaluation took: 1.953 s
total result
+---------+----------+
| Type    | Value    |
+---------+----------+
| BLEU_1  | 0.539247 |
| BLEU_2  | 0.359526 |
| BLEU_3  | 0.243342 |
| BLEU_4  | 0.158530 |
| METEOR  | 0.160432 |
| ROUGE_l | 0.372772 |
| CIDEr   | 0.359995 |
| SPICE   | 0.108029 |
| SPIDEr  | 0.234012 |
+---------+----------+
